# Annex A Execution

A notebook implementing the execution rules from KerML Annex A with PyMBE.

In [1]:
import json
import pymbe.api as pm

from pathlib import Path

from typing import Any, Collection, Dict, List, Tuple, Union

from pymbe.model import Model, Element
from pymbe.model_modification import *

from pymbe.query.metamodel_navigator import is_type_undefined_mult, \
                                    is_multiplicity_one, \
                                    is_multiplicity_specific_finite, \
                                    get_finite_multiplicity_types, \
                                    identify_connectors_one_side, \
                                    get_lower_multiplicty, \
                                    get_upper_multiplicty, \
                                    does_behavior_have_write_features, \
                                    get_most_specific_feature_type, \
                                    has_type_named

from uuid import uuid4

## Atom Metadata Load

Bring up the Atom metadata.

In [2]:
filename = "A-2-Atoms"

if not filename.endswith(".json"):
    filename += ".json"

json_file = Path(Path.cwd()) / "annex_a_data" / filename

atoms_data = pm.Model.load_from_post_file(json_file)
atoms_data

c:\users\bjorn cole\documents\github\pymbe\src\pymbe\label.py:117: UserWarning: Cannot process Expression elements yet!
  warn(f"Cannot process {expression._metatype} elements yet!")
c:\users\bjorn cole\documents\github\pymbe\src\pymbe\label.py:117: UserWarning: Cannot process MetadataAccessExpression elements yet!
  warn(f"Cannot process {expression._metatype} elements yet!")


<SysML v2 Model (C:\Users\Bjorn Cole\Documents\GitHub\pyMBE\dev_docs\core_algos\annex_a_data\A-2-Atoms.json)>

## Annex A.3.2 Without Connectors Case

In [3]:
filename = "A-3-2-WithoutConnectors"

if not filename.endswith(".json"):
    filename += ".json"

json_file = Path(Path.cwd()) / "annex_a_data" / filename

without_connectors_data = pm.Model.load_from_post_file(json_file)
without_connectors_data

<SysML v2 Model (C:\Users\Bjorn Cole\Documents\GitHub\pyMBE\dev_docs\core_algos\annex_a_data\A-3-2-WithoutConnectors.json)>

In [4]:
packages = [ele for ele in without_connectors_data.elements.values() if ele._metatype == 'Package']
packages

[WithoutConnectorsModelToBeExecuted «Package»,
 WithoutConnectorsExecution «Package»]

In [5]:
without_connectors_to_execute_classifiers = \
    [ele for ele in packages[0].throughOwningMembership if ele._metatype == 'Classifier']

In [6]:
without_connectors_to_execute_classifiers

[Bicycle «Classifier», Wheel «Classifier», BikeFork «Classifier»]

### Check for Connectors to Features

In [7]:
def is_feature_connected(feature):
    print(f"...Inspecting {feature.declaredName} for connector references.")
    if hasattr(feature, "reverseReferenceSubsetting"):
        print(f"...Found link to connector end {feature.reverseReferenceSubsetting[0]}")
        return True
    else:
        print(f"...Found no reverse edge outgoing to connector end.")
        return False

In [8]:
is_feature_connected(without_connectors_to_execute_classifiers[0].throughFeatureMembership[0])

...Inspecting rollsOn for connector references.
...Found no reverse edge outgoing to connector end.


False

### Identify Important Metatypes

In [9]:
def feature_metas():
    return {'Feature', 'Step'}

def classifier_metas():
    return {'Classifier', 'Behavior', 'Structure'}

def assoc_metas():
    return {'Association'}

def connector_metas():
    return {'Connector', 'Succession'}

### Print atoms as if in KerML Files

In [10]:
def serialize_kerml_atom(atom_to_print):
    
    atom_string = "#atom\n"
    
    if hasattr(atom_to_print, "throughUnioning"):
        atom_string = ""
    
    if atom_to_print._metatype == "Classifier":
        atom_string = atom_string + "classifier "
    elif atom_to_print._metatype == "Behavior":
        atom_string = atom_string + "behavior "
    elif atom_to_print._metatype == "Association":
        atom_string = atom_string + "association "
    elif atom_to_print._metatype == "Succession":
        atom_string = atom_string + "succession "
    elif atom_to_print._metatype == "Structure":
        atom_string = atom_string + "struct "
    elif atom_to_print._metatype == "DataType":
        atom_string = atom_string + "datatype "
    
    atom_string = atom_string + atom_to_print.declaredName
    
    if hasattr(atom_to_print, "throughSubclassification"):
        atom_string = atom_string + " specializes " + \
            ", ".join([sub.declaredName for sub in atom_to_print.throughSubclassification])
        
    if hasattr(atom_to_print, "throughUnioning"):
        atom_string = atom_string + " unions " + \
            ", ".join([sub.declaredName for sub in atom_to_print.throughUnioning])
    
    if hasattr(atom_to_print, "throughFeatureMembership"):
        atom_string = atom_string + " {\n"
        for feature in atom_to_print.throughFeatureMembership:
            if feature._metatype == "Feature":
                atom_string = atom_string + "   feature "
            elif feature._metatype == "Step":
                atom_string = atom_string + "   step "
            elif feature._metatype == "Connector":
                atom_string = atom_string + "   connector "
            elif feature._metatype == "Succession":
                atom_string = atom_string + "   succession "
                
            if hasattr(feature, "throughFeatureTyping"):
                atom_string = atom_string + feature.declaredName + " : " + feature.throughFeatureTyping[0].declaredName
            else:
                atom_string = atom_string + feature.declaredName
                
            if hasattr(feature, "throughRedefinition"):
                atom_string = atom_string + " redefines " + \
                    ", ".join([sub.declaredName for sub in feature.throughRedefinition])
                
            atom_string = atom_string + ";\n"
        atom_string = atom_string + "}\n"
    elif hasattr(atom_to_print, "throughEndFeatureMembership"):
        atom_string = atom_string + " {\n"
        for feature in atom_to_print.throughEndFeatureMembership:
            if feature._metatype == "Feature":
                atom_string = atom_string + "   end feature "
            elif feature._metatype == "Step":
                atom_string = atom_string + "   step "
            elif feature._metatype == "Connector":
                atom_string = atom_string + "   connector "
                
            if hasattr(feature, "throughFeatureTyping"):
                atom_string = atom_string + feature.declaredName + " : " + feature.throughFeatureTyping[0].declaredName + ";\n"
            else:
                atom_string = atom_string + feature.declaredName + ";\n"
        atom_string = atom_string + "}\n"
    else:
        atom_string = atom_string + ";\n"
        
    return atom_string

### Atom Generation Procedure for Steps 1 through 5

In [11]:
def atom_generation_annex_a(input_model, package_to_execute, package_to_populate):
    
    # TODO: Need to make this recursive for deeply nested features
    
    bound_features_to_atom_values_dict = {}
    
    package_to_execute_classifiers = \
        [ele for ele in package_to_execute.throughOwningMembership if ele._metatype in classifier_metas()]
    
    for classifier in package_to_execute_classifiers:
        bound_features_to_atom_values_dict_update = \
            atom_generation_annex_a_one_classifier(input_model,
                                                   package_to_execute,
                                                   package_to_populate,
                                                   classifier,
                                                   1,
                                                   None,
                                                   {})
        
        for k, values in bound_features_to_atom_values_dict_update.items():
            if k in bound_features_to_atom_values_dict:
                for v in values:
                    bound_features_to_atom_values_dict_update[k].append(v)
            else:
                bound_features_to_atom_values_dict.update({k: values})
                                                

In [12]:
def inspect_type_for_atom(input_model,
                          package_to_execute,
                          package_to_populate,
                          atom_index,
                          cf,
                          considered_type,
                          values_dict_passed):
    
    bound_features_to_atom_values_dict_update = {}
    
    new_ft_classifier = None
    
    type_name = "None"
    if considered_type is not None:
        type_name = considered_type.declaredName
        
    cf_name = cf._data.get("name") or cf._data.get("effectiveName") or cf._data.get("declaredName")
    
    if not hasattr(considered_type, "throughFeatureMembership") and not hasattr(cf, "throughFeatureMembership"):
        print(f"...Atom for {cf_name} has type {type_name} that has no further features to make atoms for.")
        
        redefining_features = []
        
        if hasattr(cf, "throughFeatureMembership"):
        
            print(f"...{cf.declaredName} has features underneath, which need consideration.")

            for cf_feat in cf.throughFeatureMembership:

                if hasattr(cf_feat, "throughFeatureValue"):

                    cf_name = cf_feat._data.get("name") or cf_feat._data.get("effectiveName") or cf_feat._data.get("declaredName")

                    print(f"...Considering feature {cf_feat} with value under {new_ft_classifier}")
                    
                if hasattr(cf_feat, "throughRedefinition"):
                    
                    redefining_features = cf_feat.throughRedefinition
                    
                    print(f"...Considering redefinitions of feature {cf_feat}")
                    
                    for redef in cf_feat.throughRedefinition:
                        if hasattr(cf_feat, "throughFeatureTyping"):
                            print(f"...Redefining {redef} with types {cf_feat.throughFeatureTyping}")

        new_ft_classifier = build_from_classifier_pattern(
            owner=package_to_populate,
            name=f"My{considered_type.declaredName}{atom_index + 1}",
            model=input_model,
            specific_fields={},
            metatype=considered_type._metatype,
            superclasses=[considered_type]
        )
        print(f"Executing step 3a. Creating atom #{atom_index + 1} to be value for {cf_name} and specializing " + 
                  f"{type_name}")

        #if is_feature_connected(cf):
        if cf._id in bound_features_to_atom_values_dict_update:
            bound_features_to_atom_values_dict_update[cf._id].append(new_ft_classifier)
        else:
            bound_features_to_atom_values_dict_update.update({cf._id: [new_ft_classifier]})
    else:
        if hasattr(cf, "throughFeatureMembership"):
            print(f"...Atom for {cf_name} has nested features leading to need for more elaboration.")
        if hasattr(considered_type, "throughFeatureMembership"):
             print(f"...Atom for {cf_name} is typed with a type {type_name} that needs more elaboration.")

        bound_features_to_atom_values_dict_update = atom_generation_annex_a_one_classifier(input_model,
                                                                                           package_to_execute,
                                                                                           package_to_populate,
                                                                                           considered_type,
                                                                                           atom_index,
                                                                                           cf,
                                                                                           values_dict_passed)
        new_ft_classifiers = bound_features_to_atom_values_dict_update[cf]
        #print(f"Seeing returned dictionary {bound_features_to_atom_values_dict_update}")
        
        if len(new_ft_classifiers) == 1:
            new_ft_classifier = new_ft_classifiers[0]
        
    
        
    return bound_features_to_atom_values_dict_update

In [13]:
def atom_generation_annex_a_one_classifier(input_model,
                                           package_to_execute,
                                           package_to_populate,
                                           base_classifier,
                                           atom_index,
                                           cf_above,
                                           values_dict):
    
    print(f"Applying atom algorithm to {base_classifier.declaredName}")
    
    bound_features_to_atom_values_dict = {}
    
    classifier = base_classifier
    
    # Step 1 - create new atom from the classifier
    base_name = classifier.declaredName
    new_classifier = build_from_classifier_pattern(
        owner=package_to_populate,
        name=f"My{base_name}",
        model=input_model,
        specific_fields={},
        metatype=classifier._metatype,
        superclasses=[classifier]
    )

    print(f"Executing step 1. Working from {base_name} to create {new_classifier.declaredName}")
    
    if cf_above is not None:
        bound_features_to_atom_values_dict.update({cf_above._id: [new_classifier]})
        print(f"...(Inside {base_classifier.declaredName}) New atom will be tracked to feature {cf_above} ({cf_above._id})")

    # Step 2 - find Features with lower multiplicity > 0 that are not connectors
    
    redefined_features = set()
    already_valued_features = set()
    
    if cf_above is not None:
        if hasattr(cf_above, "throughFeatureMembership"):
            print(f"...(Inside {base_classifier.declaredName}) Inspecting nested features from {cf_above}")

            for cf_nested in cf_above.throughFeatureMembership:
                print(f"...Found nested feature {cf_nested}")
                
                if hasattr(cf_nested, "throughRedefinition"):
                    for cf_nested_redefined in cf_nested.throughRedefinition:
                        redefined_features.add(cf_nested_redefined)
                        
                feature_values_shared = False
                    
                print(f"...Looking to see if {cf_nested} is bound to other feature values")

                if hasattr(cf_nested, "throughFeatureValue"):
                    if cf_nested.throughFeatureValue[0]._metatype == 'FeatureReferenceExpression':
                        referred_item = cf_nested.throughFeatureValue[0].throughMembership[0]
                        feature_values_shared = True

                if feature_values_shared:
                    print(f"...Taking values from {referred_item} to match to values set for {cf_nested} ({cf_nested._id})")
                    if referred_item._id in values_dict:
                        other_values = values_dict[referred_item._id]
                        print(f"...Found values {other_values} to match to values set for {cf_nested} ({cf_nested._id})")
                        already_valued_features.add(cf_nested)
                        if cf_nested._id in bound_features_to_atom_values_dict.keys():
                            bound_features_to_atom_values_dict[cf_nested._id].append(other_values)
                        else:
                            bound_features_to_atom_values_dict.update({cf_nested._id: other_values})

    if hasattr(classifier, "throughFeatureMembership") or hasattr(cf_above, "throughFeatureMembership"):
        candidate_features = []
        if hasattr(classifier, "throughFeatureMembership"):
            for ctfm in classifier.throughFeatureMembership:
                if not has_type_named(ctfm, "FeatureWritePerformance"):
                    candidate_features.append(ctfm)
        if cf_above is not None and hasattr(cf_above, "throughFeatureMembership"):
            for cffm in cf_above.throughFeatureMembership:
                if not has_type_named(cffm, "FeatureWritePerformance"):
                    candidate_features.append(cffm)
        #candidate_features = classifier.throughFeatureMembership

        # Step 3 - create Atoms to go with the Features
        
        print(f"...(Inside {base_classifier.declaredName}) Candidate owned features list is {candidate_features}")

        for cf in candidate_features:
            # TODO: Want two passes here - do only non-connectors, then a whole separate loop for connectors rather than
            # interleave as we have currently
            
            cf_name = cf._data.get("name") or cf._data.get("effectiveName") or cf._data.get("declaredName")
            
            used_feature = cf
            
            skip_base_feature = False
            
            if cf._metatype not in connector_metas():
                lm = get_lower_multiplicty(cf)
                if hasattr(cf, "throughRedefinition") and lm == -1:
                    lm = get_lower_multiplicty(cf.throughRedefinition[0])
                    print(f"...Found that {cf_name} redefines a feature with multiplicity {lm}")
                if lm > -1:
                    # need to test multiplicity
                    print(f"Executing step 2. Identified {cf_name} ({cf._id}) as a non-connector Feature. Lower multiplicity is {lm}")
                    
                    if cf in redefined_features:
                        print(f"...Discovered that {cf} ({cf._id}) is redefined by {cf.reverseRedefinition} ({cf.reverseRedefinition[0]._id})!")
                        used_feature = cf.reverseRedefinition[0]
                        skip_base_feature = True
                        
                    if has_type_named(cf, "FeatureWritePerformance"):
                        skip_base_feature = True
                        
                    if cf in already_valued_features:
                        skip_base_feature = True
                        print(f"...Found that {cf_name} ({cf._id}) already has values assigned.")
                    
                    if not skip_base_feature:
                        feature_value_atoms = []

                        used_typ = None
                        referred_item = None

                        # check to see if the value is bound to something that has already been created

                        feature_values_shared = False

                        print(f"...Looking to see if {cf} is bound to other feature values")

                        if hasattr(cf, "throughFeatureValue"):
                            if cf.throughFeatureValue[0]._metatype == 'FeatureReferenceExpression':
                                referred_item = used_feature.throughFeatureValue[0].throughMembership[0]
                                #feature_values_shared = True

                        if feature_values_shared:
                            print(f"...Taking values from {referred_item} to match to values set for {cf} ({used_feature._id})")
                            print(f"Bound values dict is currently {bound_features_to_atom_values_dict}")
                            other_values = bound_features_to_atom_values_dict[referred_item._id]
                            if cf._id in bound_features_to_atom_values_dict.keys():
                                bound_features_to_atom_values_dict[cf._id].append(other_values)
                            else:
                                bound_features_to_atom_values_dict.update({cf._id: other_values})

                        if not feature_values_shared:
                            print(f"...Generating new values for {cf} ({used_feature._id})")
                            for i in range(0, lm):

                                used_typ = get_most_specific_feature_type(used_feature)

                                #typ = used_feature.throughFeatureTyping

                                #if len(typ) > 0:
                                #    used_typ = typ[0]

                                bound_features_to_atom_values_dict_update = inspect_type_for_atom(input_model,
                                                                                                  package_to_execute,
                                                                                                  package_to_populate,
                                                                                                  i,
                                                                                                  used_feature,
                                                                                                  used_typ,
                                                                                                  bound_features_to_atom_values_dict
                                                                                                 )


                                for k, values in bound_features_to_atom_values_dict_update.items():
                                    if k in bound_features_to_atom_values_dict:
                                        for v in values:
                                            bound_features_to_atom_values_dict[k].append(v)
                                            feature_value_atoms.append(v)
                                    else:
                                        bound_features_to_atom_values_dict.update({k: values})
                                        for v in values:
                                            feature_value_atoms.append(v)

                                print(f"...(Inside {base_classifier.declaredName}) Looking for members under feature {cf_name}")

                                if hasattr(cf, "throughMembership"):
                                    feature_members = used_feature.throughMembership
                                    print(f"...Found members {feature_members}")

                                    #print(f"{bound_features_to_atom_values_dict_update}")

                    # use the covering pattern on the Feature
                    
        # Step 6 (actually comes before connectors?) - for behaviors with FeatureWritePerformances, time slice the thing with the features
    
        if does_behavior_have_write_features(base_classifier):
            print(f"Executing step 6. Identified {base_classifier} as a Behavior that executes a feature write performance.")

            if hasattr(base_classifier, "throughFeatureMembership"):

                candidate_features = base_classifier.throughFeatureMembership

                performance_step = None
                fwp_reference = None
                modified_object = None
                modified_object_type = None

                for cf in candidate_features:

                    if cf._metatype == 'Step':
                        step = cf

                        if hasattr(step, "throughFeatureTyping"):
                            candidate_types = step.throughFeatureTyping
                            for ct in candidate_types:
                                if ct.declaredName == 'FeatureWritePerformance':
                                    performance_step = cf
                                    fwp_reference = ct

                if performance_step is not None:
                    print(f"...(Inside {base_classifier.declaredName}) Step {performance_step} is feature write performance.")
                    
                    if hasattr(performance_step, "throughFeatureMembership"):
                        step_features = performance_step.throughFeatureMembership
                        for sf in step_features:
                            if hasattr(sf, "throughRedefinition"):
                                redefined_features = sf.throughRedefinition
                                for rf in redefined_features:
                                    if rf.declaredName == 'onOccurrence':
                                        # need to go to the feature that is bound in this pattern
                                        if hasattr(sf, "throughFeatureValue"):
                                            if sf.throughFeatureValue[0]._metatype == 'FeatureReferenceExpression':
                                                #print (f"...Found feature value {sf.throughFeatureValue[0].throughMembership[0]}")
                                                modified_object = sf.throughFeatureValue[0].throughMembership[0]
                                                
                                                modified_object_type = modified_object.throughFeatureTyping[0]
                                                
                                                # check for redefinition
                                                if hasattr(modified_object, "reverseRedefinition"):
                                                    modified_object = modified_object.reverseRedefinition[0]
                                                
                                                    
                    
                    modified_object_name = modified_object._data.get("name") or \
                        modified_object._data.get("effectiveName") or modified_object._data.get("declaredName")
                    
                    # generate this step specially
                    new_ps_classifier = build_from_classifier_pattern(
                        owner=package_to_populate,
                        name=f"FeatureWritePerformanceToModify{modified_object_name}",
                        model=input_model,
                        specific_fields={},
                        metatype=fwp_reference._metatype,
                        superclasses=[fwp_reference]
                    )
                    print(f"... Creating atom to be value for {performance_step} and specializing " + 
                              f"{fwp_reference}")

                    #if is_feature_connected(cf):
                    if performance_step._id in bound_features_to_atom_values_dict:
                        bound_features_to_atom_values_dict[performance_step._id].append(new_ps_classifier)
                    else:
                        bound_features_to_atom_values_dict.update({performance_step._id: [new_ps_classifier]})

                if modified_object is not None:
                    print(f"...Feature {modified_object} ({modified_object._id}) is the object of the feature write performance.")

                    if modified_object_type is not None:
                        # targeted type
                        print(f"...current bound features dictionary is {bound_features_to_atom_values_dict}")
                        modified_atoms = bound_features_to_atom_values_dict[modified_object._id]
                        # create time slices here
                        new_slice = build_from_portion_pattern(
                            owner=package_to_populate,
                            name_extension=f"TimeSliceFor{(performance_step.declaredName).title()}",
                            model=input_model,
                            classifier_to_be_portioned=modified_object_type,
                            specific_fields={}
                        )
                        new_slice_feature = build_from_feature_pattern(
                            owner=modified_atoms[0],
                            name=f"afterSlice{(performance_step.declaredName).title()}",
                            model=input_model,
                            specific_fields={},
                            feature_type=new_slice,
                            direction="",
                            metatype="Feature",
                            connector_end=False,
                        )

        for cf in candidate_features:
            # Step 4 - find Connectors with appropriate end multiplicities

            if cf._metatype in connector_metas():
                print(f"Executing step 4. Identified {cf.declaredName} as a Connector")

                # check the feature ends of the connector
                
                connector_atoms = []

                connector_ends = cf.throughEndFeatureMembership

                lm_end1 = get_lower_multiplicty(connector_ends[0])
                lm_end2 = get_lower_multiplicty(connector_ends[1])

                if lm_end1 == 1 and lm_end2 == 1:
                    print(f"Executing step 5. Identified {cf} as a Connector with 1-to-1 ends")

                    end1_connected_mult = get_lower_multiplicty(connector_ends[0].throughReferenceSubsetting[0])
                    end2_connected_mult = get_lower_multiplicty(connector_ends[1].throughReferenceSubsetting[0])

                    # check to see if the already built instances have a finite value

                    if end1_connected_mult == -1:
                        if connector_ends[0].throughReferenceSubsetting[0] in bound_features_to_atom_values_dict:
                            end1_connected_mult = len(bound_features_to_atom_values_dict[connector_ends[0].throughReferenceSubsetting[0]])
                            print(f"...Using already created atoms to set number of values for {connector_ends[0].throughReferenceSubsetting[0]}")
                    if end2_connected_mult == -1:
                        if connector_ends[1].throughReferenceSubsetting[0] in bound_features_to_atom_values_dict:
                            end2_connected_mult = len(bound_features_to_atom_values_dict[connector_ends[1].throughReferenceSubsetting[0]])
                            print(f"...Using already created atoms to set number of values for {connector_ends[1].throughReferenceSubsetting[0]}")

                    print(f"...End 1 is linked to Feature {connector_ends[0].throughReferenceSubsetting[0]}" + \
                         f" with multiplicity {end1_connected_mult}")
                    print(f"...End 2 is linked to Feature {connector_ends[1].throughReferenceSubsetting[0]}" + \
                         f" with multiplicity {end2_connected_mult}")

                    number_to_make = max(end1_connected_mult, end2_connected_mult)

                    feature_value_atoms = []

                    used_typ = None

                    for i in range(0, number_to_make):
                        print(f"Executing step 5b. Creating atom #{i + 1} to be value for {cf}")

                        typ = []
                        used_typ = None
                        used_name = cf.declaredName
                        used_metatype = "Association"
                        
                        elaboration_end = False

                        if hasattr(cf, "throughFeatureTyping"):
                            typ = cf.throughFeatureTyping

                        if len(typ) > 0:
                            used_typ = typ[0]
                            used_name = used_typ.declaredName
                            used_metatype = used_typ._metatype

                        #feature_value_atoms.append(new_ft_classifier)

                        # need to do this for Association types and also nested end features

                        ends_to_process = []

                        if hasattr(cf, "throughEndFeatureMembership"):
                            for cf_ele in cf.throughEndFeatureMembership:
                                ends_to_process.append(cf_ele)
                        if used_typ is not None and hasattr(used_typ, "throughEndFeatureMembership"):
                            for cf_ele in used_typ.throughEndFeatureMembership:
                                ends_to_process.append(cf_ele)

                        for con_end in ends_to_process:
                            print(f"...Inspecting {con_end} for connected features.")
                            if hasattr(con_end, "throughReferenceSubsetting"):
                                bound_feature = con_end.throughReferenceSubsetting[0]
                                bound_feature_type = bound_feature.throughFeatureTyping

                                print(f"...Found connected feature {bound_feature}.")

                                if len(bound_feature_type) > 0:
                                    feature_used_type = bound_feature_type[0]

                                    if bound_feature._id in bound_features_to_atom_values_dict:
                                        if (i + 1) > len(bound_features_to_atom_values_dict[bound_feature._id]):
                                            print(f"Executing step 5b (1-to-1 variant). Creating atom #{i + 1} to " +
                                                  f"be value for {con_end} and also {bound_feature}")
                                            
                                            bound_features_to_atom_values_dict_update = inspect_type_for_atom(input_model,
                                                                                              package_to_execute,
                                                                                              package_to_populate,
                                                                                              i,
                                                                                              bound_feature,
                                                                                              feature_used_type,
                                                                                              bound_features_to_atom_values_dict
                                                                                             )
                                            
                                            for k, values in bound_features_to_atom_values_dict_update.items():
                                                if k in bound_features_to_atom_values_dict:
                                                    for v in values:
                                                        bound_features_to_atom_values_dict[k].append(v)
                                                else:
                                                    bound_features_to_atom_values_dict.update({k: values})
                                            
                                    else:
                                        print(f"Executing step 5b (1-to-1 variant). Creating atom #{i + 1} to " +
                                            f"be value for {con_end} and also {bound_feature}")
                                        
                                        bound_features_to_atom_values_dict_update = inspect_type_for_atom(input_model,
                                                                                              package_to_execute,
                                                                                              package_to_populate,
                                                                                              i,
                                                                                              bound_feature,
                                                                                              feature_used_type,
                                                                                              bound_features_to_atom_values_dict            
                                                                                             )
                                            
                                        for k, values in bound_features_to_atom_values_dict_update.items():
                                            if k in bound_features_to_atom_values_dict:
                                                for v in values:
                                                    bound_features_to_atom_values_dict[k].append(v)
                                            else:
                                                bound_features_to_atom_values_dict.update({k: values})
                                        

                        if len(ends_to_process) == 2:
                            source_end = ends_to_process[0]
                            target_end = ends_to_process[1]

                            source_bound_feature = source_end.throughReferenceSubsetting[0]
                            target_bound_feature = target_end.throughReferenceSubsetting[0]
                            
                            try:
                                source_atom = bound_features_to_atom_values_dict[source_bound_feature._id][i]
                            except KeyError:
                                raise KeyError(f"...Failed to find atoms for {source_bound_feature} and connector {cf}." + \
                                               f"{bound_features_to_atom_values_dict}")
                            
                            try:
                                target_atom = bound_features_to_atom_values_dict[target_bound_feature._id][i]
                            except KeyError:
                                raise KeyError(f"...Failed to find atoms for {target_bound_feature} and connector {cf}." + \
                                               f"{bound_features_to_atom_values_dict}")

                            print(f"...Typing atom association ends from {source_atom} to {target_atom} under " + 
                                 f"{used_name}{i + 1}")

                            new_cn_classifier = build_from_binary_assoc_pattern(
                                name=f"{used_name}{i + 1}",
                                source_role_name=connector_ends[0].throughReferenceSubsetting[0].declaredName,
                                target_role_name=connector_ends[1].throughReferenceSubsetting[0].declaredName,
                                source_type=source_atom,
                                target_type=target_atom,
                                model=input_model,
                                metatype=used_metatype,
                                owner=package_to_populate,
                                specific_fields={}
                            )
                            
                            connector_atoms.append(new_cn_classifier)
                        # cover connector
                        
                    redefined_bound_feature = apply_covered_connector_pattern(
                        one_member_classifiers=connector_atoms,
                        feature_to_cover=cf,
                        type_to_apply_pattern_on=new_classifier,
                        model=input_model,
                        new_types_owner=package_to_populate,
                        covering_classifier_prefix="Values for ",
                        covering_classifier_suffix="",
                        redefining_feature_prefix="",
                        redefining_feature_suffix="",
                        metatype=cf._metatype,
                        separate_connectors=True
                    )
    
        
    for bound_feature_id in bound_features_to_atom_values_dict.keys():
        bound_feature = input_model.get_element(element_id=bound_feature_id)
        print(f"(Atom style)...Working to connect the feature {bound_feature} to generated types via " + \
              f"covering pattern with values {bound_features_to_atom_values_dict[bound_feature_id]}.")
        if classifier in bound_feature.reverseFeatureMembership or cf_above in bound_feature.reverseFeatureMembership:
            redefined_bound_feature = apply_covered_feature_pattern(
                one_member_classifiers=bound_features_to_atom_values_dict[bound_feature_id],
                feature_to_cover=bound_feature,
                type_to_apply_pattern_on=new_classifier,
                model=input_model,
                new_types_owner=package_to_populate,
                covering_classifier_prefix="Values for ",
                covering_classifier_suffix="",
                redefining_feature_prefix="",
                redefining_feature_suffix=" (Closed)",
            )
            
    
    return bound_features_to_atom_values_dict
    
    #print(f"Bound features dict: \n{bound_features_to_atom_values_dict}")

In [14]:
packages[0].throughOwningMembership

[fd3d0478-00fa-4dbb-be30-185bd3fb746a «Documentation»,
 Bicycle «Classifier»,
 Wheel «Classifier»,
 BikeFork «Classifier»]

In [15]:
atom_generation_annex_a(without_connectors_data, packages[0], packages[1])

Applying atom algorithm to Bicycle
Executing step 1. Working from Bicycle to create MyBicycle
...(Inside Bicycle) Candidate owned features list is [rollsOn: Wheel «Feature», holdsWheel: BikeFork «Feature»]
Executing step 2. Identified rollsOn (ba23b9d1-762e-4e40-8faa-8037ed7fc476) as a non-connector Feature. Lower multiplicity is 2
...Looking to see if rollsOn: Wheel «Feature» is bound to other feature values
...Generating new values for rollsOn: Wheel «Feature» (ba23b9d1-762e-4e40-8faa-8037ed7fc476)
...Atom for rollsOn has type Wheel that has no further features to make atoms for.
Executing step 3a. Creating atom #1 to be value for rollsOn and specializing Wheel
...(Inside Bicycle) Looking for members under feature rollsOn
...Atom for rollsOn has type Wheel that has no further features to make atoms for.
Executing step 3a. Creating atom #2 to be value for rollsOn and specializing Wheel
...(Inside Bicycle) Looking for members under feature rollsOn
(Atom style)...Working to connect the 

In [16]:
for item in packages[1].throughOwningMembership:
    print(serialize_kerml_atom(item))

#atom
classifier MyBicycle specializes Bicycle {
   feature rollsOn (Closed) : Values for rollsOn;
}

#atom
classifier MyWheel1 specializes Wheel;

#atom
classifier MyWheel2 specializes Wheel;

classifier Values for rollsOn unions MyWheel1, MyWheel2;

#atom
classifier MyWheel specializes Wheel;

#atom
classifier MyBikeFork specializes BikeFork;



## Annex A.3.3 One-To-One Connectors Case

In [17]:
filename = "A-3-3-OneToOneConnectors"

if not filename.endswith(".json"):
    filename += ".json"

json_file = Path(Path.cwd()) / "annex_a_data" / filename

one_2_one_connectors_data = pm.Model.load_from_post_file(json_file)
one_2_one_connectors_data

<SysML v2 Model (C:\Users\Bjorn Cole\Documents\GitHub\pyMBE\dev_docs\core_algos\annex_a_data\A-3-3-OneToOneConnectors.json)>

In [18]:
packages = [ele for ele in one_2_one_connectors_data.elements.values() if ele._metatype == 'Package']
packages

[Atoms «Package»,
 WithoutConnectorsModelToBeExecuted «Package»,
 OneToOneConnectorsModelToBeExecuted «Package»,
 OneToOneConnectorsExecution «Package»]

In [19]:
atom_generation_annex_a(one_2_one_connectors_data, packages[2], packages[3])

Applying atom algorithm to Bicycle
Executing step 1. Working from Bicycle to create MyBicycle
...(Inside Bicycle) Candidate owned features list is [<Connector([rollsOn: Wheel «Feature»] ←→ [holdsWheel: BikeFork «Feature»])>, rollsOn: Wheel «Feature», holdsWheel: BikeFork «Feature»]
Executing step 2. Identified rollsOn (fb0065c5-0cdc-4f61-be2b-29c9083f5efe) as a non-connector Feature. Lower multiplicity is 2
...Looking to see if rollsOn: Wheel «Feature» is bound to other feature values
...Generating new values for rollsOn: Wheel «Feature» (fb0065c5-0cdc-4f61-be2b-29c9083f5efe)
...Atom for rollsOn has type Wheel that has no further features to make atoms for.
Executing step 3a. Creating atom #1 to be value for rollsOn and specializing Wheel
...(Inside Bicycle) Looking for members under feature rollsOn
...Atom for rollsOn has type Wheel that has no further features to make atoms for.
Executing step 3a. Creating atom #2 to be value for rollsOn and specializing Wheel
...(Inside Bicycle) Loo

In [20]:
for item in packages[3].throughOwningMembership:
    print(serialize_kerml_atom(item))

#atom
classifier MyBicycle specializes Bicycle {
   connector fixWheel0 : BikeWheelFixed1;
   connector fixWheel1 : BikeWheelFixed2;
   feature rollsOn (Closed) : Values for rollsOn;
   feature holdsWheel (Closed) : Values for holdsWheel;
}

#atom
classifier MyWheel1 specializes Wheel;

#atom
classifier MyWheel2 specializes Wheel;

#atom
classifier MyBikeFork1 specializes BikeFork;

#atom
association BikeWheelFixed1 {
   end feature rollsOn : MyWheel1;
   end feature holdsWheel : MyBikeFork1;
}

#atom
classifier MyBikeFork2 specializes BikeFork;

#atom
association BikeWheelFixed2 {
   end feature rollsOn : MyWheel2;
   end feature holdsWheel : MyBikeFork2;
}

classifier Values for rollsOn unions MyWheel1, MyWheel2;

classifier Values for holdsWheel unions MyBikeFork1, MyBikeFork2;



## Annex A.3.6 Timing for Behaviors, Sequences

In [21]:
filename = "A-3-6-Sequences"

if not filename.endswith(".json"):
    filename += ".json"

json_file = Path(Path.cwd()) / "annex_a_data" / filename

sequences_data = pm.Model.load_from_post_file(json_file)
sequences_data

<SysML v2 Model (C:\Users\Bjorn Cole\Documents\GitHub\pyMBE\dev_docs\core_algos\annex_a_data\A-3-6-Sequences.json)>

In [22]:
packages = [ele for ele in sequences_data.elements.values() if ele._metatype == 'Package']
packages

[SequencesModelToBeExecuted «Package», SequencesExecution «Package»]

In [23]:
packages[0].ownedElement

[38b62785-6ce9-40d6-87a8-e0f556581f56 «Documentation»,
 Manufacture «Behavior»,
 Paint «Behavior»,
 Dry «Behavior»,
 Ship «Behavior»]

In [24]:
atom_generation_annex_a(sequences_data, packages[0], packages[1])

Applying atom algorithm to Manufacture
Executing step 1. Working from Manufacture to create MyManufacture
...(Inside Manufacture) Candidate owned features list is [paint: Paint «Step», dry: Dry «Step», <Succession([paint: Paint «Step»] ←→ [dry: Dry «Step»])>, ship: Ship «Step», <Succession([dry: Dry «Step»] ←→ [ship: Ship «Step»])>]
Executing step 2. Identified paint (25ee82c7-0485-4fb8-8b72-bbd960872469) as a non-connector Feature. Lower multiplicity is 1
...Looking to see if paint: Paint «Step» is bound to other feature values
...Generating new values for paint: Paint «Step» (25ee82c7-0485-4fb8-8b72-bbd960872469)
...Atom for paint has type Paint that has no further features to make atoms for.
Executing step 3a. Creating atom #1 to be value for paint and specializing Paint
...(Inside Manufacture) Looking for members under feature paint
Executing step 4. Identified p_before_d as a Connector
Executing step 5. Identified <Succession([paint: Paint «Step»] ←→ [dry: Dry «Step»])> as a Conne

In [25]:
for item in packages[1].throughOwningMembership:
    print(serialize_kerml_atom(item))

#atom
behavior MyManufacture specializes Manufacture {
   succession p_before_d0 : p_before_d1;
   succession d_before_s0 : d_before_s1;
   feature paint (Closed) : MyPaint1;
   feature dry (Closed) : MyDry1;
   feature ship (Closed) : MyShip1;
}

#atom
behavior MyPaint1 specializes Paint;

#atom
behavior MyDry1 specializes Dry;

#atom
association p_before_d1 {
   end feature paint : MyPaint1;
   end feature dry : MyDry1;
}

#atom
behavior MyShip1 specializes Ship;

#atom
association d_before_s1 {
   end feature dry : MyDry1;
   end feature ship : MyShip1;
}

#atom
behavior MyPaint specializes Paint;

#atom
behavior MyDry specializes Dry;

#atom
behavior MyShip specializes Ship;



In [26]:
packages[1].throughOwningMembership[0].throughFeatureMembership

[p_before_d0: p_before_d1 «Succession»,
 d_before_s0: d_before_s1 «Succession»,
 paint (Closed): MyPaint1 «Feature»,
 dry (Closed): MyDry1 «Feature»,
 ship (Closed): MyShip1 «Feature»]

## Annex A.3.8 Feature Value Changes

In [27]:
filename = "A-3-8-ChangingFeatureValues"

if not filename.endswith(".json"):
    filename += ".json"

json_file = Path(Path.cwd()) / "annex_a_data" / filename

values_data = pm.Model.load_from_post_file(json_file)
values_data

<SysML v2 Model (C:\Users\Bjorn Cole\Documents\GitHub\pyMBE\dev_docs\core_algos\annex_a_data\A-3-8-ChangingFeatureValues.json)>

In [28]:
packages = [ele for ele in values_data.elements.values() if ele._metatype == 'Package']
packages

[ChangingFeatureValuesModelToBeExecuted «Package»,
 ChangingFeatureValuesExecution «Package»]

In [29]:
#atom_generation_annex_a(values_data, packages[0], packages[1])

In [30]:
packages[0].throughOwningMembership

[72f33ab9-7786-4ddd-962b-3dba4f1cf004 «Documentation»,
 Manufacture «Behavior»,
 Product «Structure»,
 Paint «Behavior»,
 Dry «Behavior»,
 Ship «Behavior»]

In [31]:
packages[0].throughOwningMembership[3].throughFeatureMembership[1].throughFeatureMembership[0]

onOccurrence: Product «Feature»

In [32]:
%pdb

Automatic pdb calling has been turned ON


In [33]:
atom_generation_annex_a_one_classifier(values_data, packages[0], packages[1], packages[0].throughOwningMembership[1], 0, None, {})

Applying atom algorithm to Manufacture
Executing step 1. Working from Manufacture to create MyManufacture
...(Inside Manufacture) Candidate owned features list is [objectToFinish: Product «Feature», paint: Paint «Step», dry: Dry «Step», <Succession([paint: Paint «Step»] ←→ [dry: Dry «Step»])>, ship: Ship «Step», <Succession([dry: Dry «Step»] ←→ [ship: Ship «Step»])>]
Executing step 2. Identified objectToFinish (62e2dc0b-a9e9-4097-aa48-209ef95bb612) as a non-connector Feature. Lower multiplicity is 1
...Looking to see if objectToFinish: Product «Feature» is bound to other feature values
...Generating new values for objectToFinish: Product «Feature» (62e2dc0b-a9e9-4097-aa48-209ef95bb612)
...Atom for objectToFinish is typed with a type Product that needs more elaboration.
Applying atom algorithm to Product
Executing step 1. Working from Product to create MyProduct
...(Inside Product) New atom will be tracked to feature objectToFinish: Product «Feature» (62e2dc0b-a9e9-4097-aa48-209ef95bb61

{'62e2dc0b-a9e9-4097-aa48-209ef95bb612': [MyProduct «Structure»],
 '85f9535c-76d9-48c7-b457-d7b650e32947': [MyBoolean1 «DataType»,
  MyBoolean1 «DataType»,
  MyBoolean1 «DataType»],
 '737cde93-27cf-42a8-af4e-13aae3f0b15a': [MyBoolean1 «DataType»,
  MyBoolean1 «DataType»,
  MyBoolean1 «DataType»],
 '27cd02a4-9e95-404a-b7d1-67f31a206cf0': [MyBoolean1 «DataType»,
  MyBoolean1 «DataType»,
  MyBoolean1 «DataType»],
 'dd76a0b4-2264-4cea-bd0f-69ebe307e1e4': [MyPaint «Behavior»],
 '952c4a56-d3c3-448a-b01a-2b2b70179ec2': [MyProduct «Structure»],
 'cd6c3ed6-81f5-43bb-90b6-acaec9a69b7d': [FeatureWritePerformanceToModifyobjectToPaint «Behavior»],
 '95bb916d-2254-4152-be1e-b767291341c7': [MyFeatureWritePerformance «Behavior»],
 'f7b380a0-f368-4617-b15a-45817338f786': [MyProduct «Structure»],
 '8a490f08-071c-438f-95ee-10a8193b3262': [MyProduct «Structure»],
 '6c969760-2ff1-4f93-b4a3-b8d7852b6563': [MyBoolean1 «DataType»],
 '23a88d58-03fd-42a1-abce-b40e87f2f3cd': [MyDry «Behavior»],
 '65c2a9ca-a694-4

In [34]:
for item in packages[1].throughOwningMembership:
    print(serialize_kerml_atom(item))

#atom
behavior MyManufacture specializes Manufacture {
   succession p_before_d0 : p_before_d1;
   succession d_before_s0 : d_before_s1;
   feature objectToFinish (Closed) : MyProduct;
   feature paint (Closed) : MyPaint;
   feature dry (Closed) : MyDry;
   feature ship (Closed) : MyShip;
}

#atom
struct MyProduct specializes Product {
   feature isPainted (Closed) : MyBoolean1;
   feature isDry (Closed) : MyBoolean1;
   feature isShipped (Closed) : MyBoolean1;
   feature afterSlicePainted : ProductTimeSliceForPainted;
   feature afterSliceDried : ProductTimeSliceForDried;
   feature afterSliceShipped : ProductTimeSliceForShipped;
}

#atom
datatype MyBoolean1 specializes Boolean;

#atom
datatype MyBoolean1 specializes Boolean;

#atom
datatype MyBoolean1 specializes Boolean;

#atom
behavior MyPaint specializes Paint {
   succession p_before_p0 : p_before_p1;
   feature objectToPaint (Closed) : MyProduct;
   feature painted (Closed) : FeatureWritePerformanceToModifyobjectToPaint;
   feat